In [56]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
tf.version


<module 'tensorflow._api.v2.version' from 'c:\\Users\\pc\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>

$LINEAR...REGRESSION$

In [57]:
import csv

In [58]:
import tensorflow.compat.v2.feature_column as tf1
# need this  for feature selection exclusivly for tensorflow
# "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

In [59]:
import csv as c

file = open('eval.csv', 'a', newline="")
obj = c.writer(file)
obj.writerow(['','female', 23.9, 3, 0, 8.56, 'Second', 'unknown', 'Southampton', 'y'])
file.close()

dataf=pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
testf=pd.read_csv("eval.csv")

label=dataf.pop('survived')
labelt=testf.pop('survived')


testf.shape


(266, 9)

In [60]:
print(testf.loc[264])
testf.shape
(264, 9)

sex                        female
age                          23.9
n_siblings_spouses              3
parch                           0
fare                         8.56
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 264, dtype: object


(264, 9)

In [61]:
dataf.shape
dataf.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [62]:
# our dataset has a lot of categorical data that is non -numeric data so we \
# need to make our features in numeric format#
# so we need to use keras modeling layer


categorical = ['sex', 'class', 'deck', 'embark_town', 'alone']
numeric = ['age', 'n_siblings_spouses', 'parch', 'fare']

features=[]

for i in categorical:
    vocab=dataf[i].unique()
    print(i,":",vocab)
    encod = tf.feature_column.categorical_column_with_vocabulary_list(i,vocab)
    features.append(encod)

for i in numeric:
    encod = tf.feature_column.numeric_column(i,dtype=tf.float64)
    features.append(encod)

features

sex : ['male' 'female']
class : ['Third' 'First' 'Second']
deck : ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town : ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone : ['n' 'y']


[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='parch', shape=(1,)

In [63]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():  # inner function, this will be returned
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
        if shuffle:
            ds = ds.shuffle(1000)  # randomize order of data
            ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
        return ds  # return a batch of the dataset
    return input_function  # return a function object for use

train_input_fn = make_input_fn(dataf,label)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(testf,labelt, num_epochs=10, shuffle=True)

In [64]:
linear_est = tf.estimator.LinearClassifier(feature_columns=features)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\pc\\AppData\\Local\\Temp\\tmpy4hflwlb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [65]:
linear_est.train(train_input_fn)  # train
# result = linear_est.evaluate(eval_input_fn)
# print(result['accuracy'])


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\pc\AppData\Local\Temp\tmpy4hflwlb\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 542.918
INFO:tensorflow:loss = 0.6308008, step = 100 (0.185 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 200...
INFO:tensorflow:Saving checkpoints for 200 into C:\Users\pc\AppData\Local\Temp\tmpy4hflwlb\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 200...
INFO:tensorflow:Loss for final step: 0.28251037.


In [66]:
pred_dicts = list(linear_est.predict(eval_input_fn))
# probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

x=testf.shape[0]-1
print(testf.loc[x],sep=" ")
print(label.loc[x],sep=" ")
print(pred_dicts[x]['probabilities'][0],"% chance of survival")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\pc\AppData\Local\Temp\tmpy4hflwlb\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                          23.9
n_siblings_spouses              3
parch                           0
fare                         8.56
class                      Second
deck                      unknown
embark_town           Southampton
alone                           y
Name: 265, dtype: object
1
0.85759795 % chance of survival


In [93]:
def make_input_fn(data_df, batch_size=32):

    def input_function():  # inner function, this will be returned
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df))).batch(
            batch_size
        )  # create tf.data.Dataset object with data and its label

        return ds  # return a batch of the dataset

    return input_function  # return a function object for use


features = [
    'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck',
    'embark_town', 'alone'
]
# predict = dict(['male', 26.9, 3, 0, 12.3, 'First', 'unknown', 'Southampton', 'n'])

predict = {
    'sex': 'male',
    'age': 25,
    'n_siblings_spouses': 5,
    'parch': 0,
    'fare': 0,
    'class': 'First',
    'deck': 'unknown',
    'embark_town': 'Southampton',
    'alone': 'n'
}
# print("Please type numeric values as prompted.")
# for feature in features:
#     val = input(feature + ": ")
#     if val.isnumeric():
#         predict[feature] = [float(val)]
#     if val.isalpha():
#         predict[feature] = [str(val)]

prediction = linear_est.predict(make_input_fn(predict))
prediction


<generator object Estimator.predict at 0x000002B6B19D6570>